The code you provided can be optimized in several ways. Here, I propose several improvements:

    Use NumPy vectorized operations instead of loops where possible.
    Precompute the cumulative sum of wealth to speed up the Gini calculation.
    Remove unnecessary function calls and inline the logic.

Here's the optimized version of the code:

Bueno, no funciona porque quiso vectorizar las transacciones pero esto puede\
generar errores, ya que en MCS un agente puede tener más de una transacción\
pero puede quedar sin riqueza antes de completarlas todas. Igual mejoró la función\
get_gini . is_valid.

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
class FastYSmodel:
    def __init__(self, n_agentes, w_min=1e-15, f=0, every=1):
        self.N = n_agentes
        self.w_min = w_min
        self.every = every
        self.n = np.random.rand(self.N, 2)
        self.n[:, 0] = self.n[:, 0]/(np.sum(self.n[:, 0]))
        self.gini = []
        self.gini.append(self.get_gini())
        self.f = f

    def get_opponents(self):
        random_array = np.random.randint(0, self.N, self.N)
        self.indices = np.arange(0, self.N)
        random_array = np.where(random_array == self.indices, 
                                (random_array + 1) % self.N, 
                                random_array)
        return random_array

    def is_valid(self, i, j):
        valid = (self.n[i, 0] > self.w_min) & (self.n[j, 0] > self.w_min)
        self.n[i, 0] = np.where(self.n[i, 0] < self.w_min, 0, self.n[i, 0])
        self.n[j, 0] = np.where(self.n[j, 0] < self.w_min, 0, self.n[j, 0])
        return valid

    def get_dw(self, i, j):
        return np.minimum(self.n[i,0]*self.n[i,1], self.n[j,0]*self.n[j,1])

    def get_gini(self):
        w = np.sort(self.n[:, 0])[::-1]
        p_cumsum = np.cumsum(w) / np.sum(w)
        B = np.sum(p_cumsum) / self.N
        return 1 + 1 / self.N - 2 * B

    def update_wealth(self, i, j, dw):
        self.n[i, 0] += dw
        self.n[j, 0] -= dw

    def choose_winner(self, i, j):
        p = .5 + self.f*((self.n[j,0] - self.n[i,0])/
                         (self.n[i,0] + self.n[j,0]))
        winner = np.random.choice([i, j], p=[p, 1-p])
        return winner

    def MCS(self, steps):
        for mcs in range(steps):
            j = self.get_opponents()

            valid = self.is_valid(self.indices, j)
            dw = self.get_dw(self.indices, j)
            winner = self.choose_winner(self.indices, j)
            dw = np.where(winner == self.indices, dw, -dw)
            self.update_wealth(self.indices, j, dw)

            if ((mcs + 1) % 10 == 0):
                self.gini.append(self.get_gini())


In [3]:
model = FastYSmodel(1000, w_min=1e-14, f=.1, every=10)
model.MCS(1000)

ValueError: a must be 1-dimensional